In [1]:
# noexport

!typech efficacy_utils.ipynb

[NbConvertApp] Converting notebook efficacy_utils.ipynb to python
retention_utils.py:398: error: Name 'get_ipython' is not defined
efficacy_utils.py:40: error: Name 'get_condition_to_installs_for_random_assignment_abtest' already defined on line 27
efficacy_utils.py:159: error: Name 'get_ipython' is not defined
efficacy_utils.py:163: error: Name 'get_ipython' is not defined
efficacy_utils.py:180: error: Name 'get_ipython' is not defined
efficacy_utils.py:198: error: Name 'condition_to_mlengths' is not defined


In [2]:
from retention_utils import *

<IPython.core.display.Javascript object>

In [3]:
import jsonmemo as jsonmemo_module
jsonmemo_module.set_lowmem(True)

In [4]:
#installs_with_experiment_vars = get_installs_with_experiment_vars()

In [5]:
#print(installs_with_experiment_vars[0])

In [6]:


def get_condition_to_installs_for_random_assignment_abtest():
  abtest_name = 'difficulty_selection_screen'
  groups = ['survey_nochoice_nothing', 'survey_nochoice_easy', 'survey_nochoice_medium', 'survey_nochoice_hard']
  condition_to_installs = get_conditions_to_install_list_in_abtest_unstrict(abtest_name)
  for k in list(condition_to_installs.keys()):
    if k not in groups:
      del condition_to_installs[k]
  return condition_to_installs


In [7]:


def get_condition_to_installs_for_random_assignment_abtest():
  abtest_name = 'difficulty_selection_screen_and_choose_difficulty_frequency'
  groups = ['survey', 'nodefault_forcedchoice_userchoice', 'survey_nochoice_nothing', 'survey_nochoice_easy', 'survey_nochoice_medium', 'survey_nochoice_hard']
  condition_to_installs = get_conditions_to_install_list_in_abtest_unstrict(abtest_name)
  for k in list(condition_to_installs.keys()):
    if k not in groups:
      del condition_to_installs[k]
  return condition_to_installs


In [8]:

#for install in installs_with_experiment_vars:
@msgpackmemo1arg
def get_domain_to_intervention_to_session_lengths_for_install(install):
  seconds_on_domain_per_session = get_collection_for_install(install, 'synced:seconds_on_domain_per_session')
  interventions_active_for_domain_and_session = get_collection_for_install(install, 'synced:interventions_active_for_domain_and_session')
  
  domain_to_session_to_intervention = {}
  for x in interventions_active_for_domain_and_session:
    if x.get('val') is None:
      continue
    interventions_active = json.loads(x['val'])
    if len(interventions_active) == 0:
      continue
    intervention_name = interventions_active[0]
    domain = x['key']
    session_id = x['key2']
    if domain not in domain_to_session_to_intervention:
      domain_to_session_to_intervention[domain] = {}
    domain_to_session_to_intervention[domain][session_id] = intervention_name
  
  domain_to_intervention_to_session_lengths = {}
  for x in seconds_on_domain_per_session:
    if 'key' not in x:
      print('missing key in seconds_on_domain_per_session')
      print(x)
      continue
    domain = x['key']
    session_id = x['key2']
    time_spent = x['val']
    if domain not in domain_to_session_to_intervention:
      continue
    intervention_name = domain_to_session_to_intervention[domain].get(session_id)
    if intervention_name is None:
      continue
    if domain not in domain_to_intervention_to_session_lengths:
      domain_to_intervention_to_session_lengths[domain] = {}
    if intervention_name not in domain_to_intervention_to_session_lengths[domain]:
      domain_to_intervention_to_session_lengths[domain][intervention_name] = []
    domain_to_intervention_to_session_lengths[domain][intervention_name].append(time_spent)
  return domain_to_intervention_to_session_lengths
  

In [9]:
# noexport

for x in get_collection_names():
  if 'per_day' in x and 'visits' not in x:
    print(x)
    break

edceda6041ef668213a96ba5_synced:seconds_on_domain_per_day


In [10]:
@msgpackmemo1arg
def get_epoch_to_domain_to_time_spent(install):
  seconds_on_domain_per_day = get_collection_for_install(install, 'synced:seconds_on_domain_per_day')
  output = {}
  for x in seconds_on_domain_per_day:
    if 'key' not in x:
      print('missing key in seconds_on_domain_per_day')
      print(x)
      continue
    domain = x['key']
    epoch = x['key2']
    seconds = x['val']
    if epoch not in output:
      output[epoch] = {}
    output[epoch][domain] = seconds
  return output

In [11]:
# noexport

condition_to_installs = get_condition_to_installs_for_random_assignment_abtest()


In [12]:
# noexport

for condition,installs in condition_to_installs.items():
  for install in installs:
    print(sorted(get_epoch_to_domain_to_time_spent(install).keys()))
    break

exception in msgpackmemo1arg for file 2019_07_30/get_epoch_to_domain_to_time_spent/00225f3e274bb40be3e8d77d.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_epoch_to_domain_to_time_spent/00225f3e274bb40be3e8d77d.msgpack'
performing computation 2019_07_30/get_epoch_to_domain_to_time_spent/00225f3e274bb40be3e8d77d.msgpack for arg 00225f3e274bb40be3e8d77d
exception in msgpackmemo1arg for file 2019_07_30/get_collection_items/0742ec8e619d030db1aa8254_synced:seconds_on_domain_per_day.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_collection_items/0742ec8e619d030db1aa8254_synced:seconds_on_domain_per_day.msgpack'
performing computation 2019_07_30/get_collection_items/0742ec8e619d030db1aa8254_synced:seconds_on_domain_per_day.msgpack for arg 0742ec8e619d030db1aa8254_synced:seconds_on_domain_per_day
done with computation 2019_07_30/get_collection_items/0742ec8e619d030db1aa8254_synced:seconds_on_domain_per_day.msgpack
done with computation 2019_07_30/get_epoch_to_domain_to_time_spent/00225f3e274bb40be3

In [13]:
import math

def make_domain_to_daily_time_dataframe():
  output = []
  for condition,installs in condition_to_installs.items():
    #condition_to_lengths[condition] = []
    for install in installs:
      for epoch,domain_to_time_spent in get_epoch_to_domain_to_time_spent(install).items():
        total_time_spent = sum(domain_to_time_spent.values())
        output.append({
          'user': install,
          'epoch': epoch,
          'time': total_time_spent,
          'logtime': math.log(total_time_spent),
          'condition': condition,
        })
#         for domain,total_time_spent in  domain_to_time_spent.items():
#           output.append({
#             'domain': domain,
#             'user': install,
#             'epoch': epoch,
#             'time': total_time_spent,
#             'logtime': math.log(total_time_spent),
#             'condition': condition,
#           })
  return to_dataframe(output)

In [14]:
df = make_domain_to_daily_time_dataframe()

exception in msgpackmemo1arg for file 2019_07_30/get_epoch_to_domain_to_time_spent/0c5737eaf60340530863e305.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_epoch_to_domain_to_time_spent/0c5737eaf60340530863e305.msgpack'
performing computation 2019_07_30/get_epoch_to_domain_to_time_spent/0c5737eaf60340530863e305.msgpack for arg 0c5737eaf60340530863e305
done with computation 2019_07_30/get_epoch_to_domain_to_time_spent/0c5737eaf60340530863e305.msgpack
exception in msgpackmemo1arg for file 2019_07_30/get_epoch_to_domain_to_time_spent/0da1764f2f4b0a547b2f955f.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_epoch_to_domain_to_time_spent/0da1764f2f4b0a547b2f955f.msgpack'
performing computation 2019_07_30/get_epoch_to_domain_to_time_spent/0da1764f2f4b0a547b2f955f.msgpack for arg 0da1764f2f4b0a547b2f955f
exception in msgpackmemo1arg for file 2019_07_30/get_collection_items/c451ecbac6f47fd22a20c48c_synced:seconds_on_domain_per_day.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_collection_items/c451ecba

done with computation 2019_07_30/get_epoch_to_domain_to_time_spent/4210c9ad2c8d826b4240e589.msgpack
exception in msgpackmemo1arg for file 2019_07_30/get_epoch_to_domain_to_time_spent/44a79b99de35752f235f8c14.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_epoch_to_domain_to_time_spent/44a79b99de35752f235f8c14.msgpack'
performing computation 2019_07_30/get_epoch_to_domain_to_time_spent/44a79b99de35752f235f8c14.msgpack for arg 44a79b99de35752f235f8c14
done with computation 2019_07_30/get_epoch_to_domain_to_time_spent/44a79b99de35752f235f8c14.msgpack
exception in msgpackmemo1arg for file 2019_07_30/get_epoch_to_domain_to_time_spent/4e6a6cef78938183dbeb00a9.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_epoch_to_domain_to_time_spent/4e6a6cef78938183dbeb00a9.msgpack'
performing computation 2019_07_30/get_epoch_to_domain_to_time_spent/4e6a6cef78938183dbeb00a9.msgpack for arg 4e6a6cef78938183dbeb00a9
exception in msgpackmemo1arg for file 2019_07_30/get_collection_items/bcd2ed106e1031eac800cd6b_syn

done with computation 2019_07_30/get_collection_items/a2181ec6d5e09584ae951010_synced:seconds_on_domain_per_day.msgpack
done with computation 2019_07_30/get_epoch_to_domain_to_time_spent/6f67ccf61109696e4aa121af.msgpack
exception in msgpackmemo1arg for file 2019_07_30/get_epoch_to_domain_to_time_spent/7202867f60e62186e71fd1ef.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_epoch_to_domain_to_time_spent/7202867f60e62186e71fd1ef.msgpack'
performing computation 2019_07_30/get_epoch_to_domain_to_time_spent/7202867f60e62186e71fd1ef.msgpack for arg 7202867f60e62186e71fd1ef
exception in msgpackmemo1arg for file 2019_07_30/get_collection_items/1a3852aa0a13aa0e5388ccd6_synced:seconds_on_domain_per_day.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_collection_items/1a3852aa0a13aa0e5388ccd6_synced:seconds_on_domain_per_day.msgpack'
performing computation 2019_07_30/get_collection_items/1a3852aa0a13aa0e5388ccd6_synced:seconds_on_domain_per_day.msgpack for arg 1a3852aa0a13aa0e5388ccd6_synced:seconds_on_

done with computation 2019_07_30/get_collection_items/de6a908835401dcaf037cc95_synced:seconds_on_domain_per_day.msgpack
done with computation 2019_07_30/get_epoch_to_domain_to_time_spent/a99d3dda7e8757513429395b.msgpack
exception in msgpackmemo1arg for file 2019_07_30/get_epoch_to_domain_to_time_spent/ae9248b9afb7fcc69bfefd74.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_epoch_to_domain_to_time_spent/ae9248b9afb7fcc69bfefd74.msgpack'
performing computation 2019_07_30/get_epoch_to_domain_to_time_spent/ae9248b9afb7fcc69bfefd74.msgpack for arg ae9248b9afb7fcc69bfefd74
exception in msgpackmemo1arg for file 2019_07_30/get_collection_items/98909f5712e78bb9c6645fd3_synced:seconds_on_domain_per_day.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_collection_items/98909f5712e78bb9c6645fd3_synced:seconds_on_domain_per_day.msgpack'
performing computation 2019_07_30/get_collection_items/98909f5712e78bb9c6645fd3_synced:seconds_on_domain_per_day.msgpack for arg 98909f5712e78bb9c6645fd3_synced:seconds_on_

done with computation 2019_07_30/get_collection_items/f41b7ced0db1cfb53fa76b23_synced:seconds_on_domain_per_day.msgpack
done with computation 2019_07_30/get_epoch_to_domain_to_time_spent/c46b20ff736164646ca7bbb4.msgpack
exception in msgpackmemo1arg for file 2019_07_30/get_epoch_to_domain_to_time_spent/c5d09bc2b691d1b34f10a7b9.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_epoch_to_domain_to_time_spent/c5d09bc2b691d1b34f10a7b9.msgpack'
performing computation 2019_07_30/get_epoch_to_domain_to_time_spent/c5d09bc2b691d1b34f10a7b9.msgpack for arg c5d09bc2b691d1b34f10a7b9
exception in msgpackmemo1arg for file 2019_07_30/get_collection_items/5647ca7d11471cb6edc8b708_synced:seconds_on_domain_per_day.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_collection_items/5647ca7d11471cb6edc8b708_synced:seconds_on_domain_per_day.msgpack'
performing computation 2019_07_30/get_collection_items/5647ca7d11471cb6edc8b708_synced:seconds_on_domain_per_day.msgpack for arg 5647ca7d11471cb6edc8b708_synced:seconds_on_

...

done with computation 2019_07_30/get_collection_items/d2b86764939576853b44f25d_synced:seconds_on_domain_per_day.msgpack
done with computation 2019_07_30/get_epoch_to_domain_to_time_spent/1382be0f11d88bf51882e6dd.msgpack
exception in msgpackmemo1arg for file 2019_07_30/get_epoch_to_domain_to_time_spent/163a274ab35687ed2cbb37d4.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_epoch_to_domain_to_time_spent/163a274ab35687ed2cbb37d4.msgpack'
performing computation 2019_07_30/get_epoch_to_domain_to_time_spent/163a274ab35687ed2cbb37d4.msgpack for arg 163a274ab35687ed2cbb37d4
exception in msgpackmemo1arg for file 2019_07_30/get_collection_items/b057b25faa934543b5bc4cbb_synced:seconds_on_domain_per_day.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_collection_items/b057b25faa934543b5bc4cbb_synced:seconds_on_domain_per_day.msgpack'
performing computation 2019_07_30/get_collection_items/b057b25faa934543b5bc4cbb_synced:seconds_on_domain_per_day.msgpack for arg b057b25faa934543b5bc4cbb_synced:seconds_on_

done with computation 2019_07_30/get_collection_items/fd8a9f60b470486a5410b11d_synced:seconds_on_domain_per_day.msgpack
done with computation 2019_07_30/get_epoch_to_domain_to_time_spent/3c6fbab8068fab8845f47ba1.msgpack
exception in msgpackmemo1arg for file 2019_07_30/get_epoch_to_domain_to_time_spent/3e5a4c953f5181b3e1ad8d96.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_epoch_to_domain_to_time_spent/3e5a4c953f5181b3e1ad8d96.msgpack'
performing computation 2019_07_30/get_epoch_to_domain_to_time_spent/3e5a4c953f5181b3e1ad8d96.msgpack for arg 3e5a4c953f5181b3e1ad8d96
exception in msgpackmemo1arg for file 2019_07_30/get_collection_items/42b287f69f25cb24a814a507_synced:seconds_on_domain_per_day.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_collection_items/42b287f69f25cb24a814a507_synced:seconds_on_domain_per_day.msgpack'
performing computation 2019_07_30/get_collection_items/42b287f69f25cb24a814a507_synced:seconds_on_domain_per_day.msgpack for arg 42b287f69f25cb24a814a507_synced:seconds_on_

done with computation 2019_07_30/get_collection_items/41d164429f11ab03c5d257b5_synced:seconds_on_domain_per_day.msgpack
done with computation 2019_07_30/get_epoch_to_domain_to_time_spent/72c593b5e280488490ae533c.msgpack
exception in msgpackmemo1arg for file 2019_07_30/get_epoch_to_domain_to_time_spent/7588a68b8f3d4375d42c386f.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_epoch_to_domain_to_time_spent/7588a68b8f3d4375d42c386f.msgpack'
performing computation 2019_07_30/get_epoch_to_domain_to_time_spent/7588a68b8f3d4375d42c386f.msgpack for arg 7588a68b8f3d4375d42c386f
exception in msgpackmemo1arg for file 2019_07_30/get_collection_items/87e45d043058edfe00698bcf_synced:seconds_on_domain_per_day.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_collection_items/87e45d043058edfe00698bcf_synced:seconds_on_domain_per_day.msgpack'
performing computation 2019_07_30/get_collection_items/87e45d043058edfe00698bcf_synced:seconds_on_domain_per_day.msgpack for arg 87e45d043058edfe00698bcf_synced:seconds_on_

done with computation 2019_07_30/get_epoch_to_domain_to_time_spent/9e44bcd734953ff22e24f91e.msgpack
exception in msgpackmemo1arg for file 2019_07_30/get_epoch_to_domain_to_time_spent/9f145b278a64c6232374614c.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_epoch_to_domain_to_time_spent/9f145b278a64c6232374614c.msgpack'
performing computation 2019_07_30/get_epoch_to_domain_to_time_spent/9f145b278a64c6232374614c.msgpack for arg 9f145b278a64c6232374614c
exception in msgpackmemo1arg for file 2019_07_30/get_collection_items/fb5f5d5f2e199b8aa2920097_synced:seconds_on_domain_per_day.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_collection_items/fb5f5d5f2e199b8aa2920097_synced:seconds_on_domain_per_day.msgpack'
performing computation 2019_07_30/get_collection_items/fb5f5d5f2e199b8aa2920097_synced:seconds_on_domain_per_day.msgpack for arg fb5f5d5f2e199b8aa2920097_synced:seconds_on_domain_per_day
done with computation 2019_07_30/get_collection_items/fb5f5d5f2e199b8aa2920097_synced:seconds_on_domain_p

done with computation 2019_07_30/get_epoch_to_domain_to_time_spent/d9c3505b20f8be8fd31486f4.msgpack
exception in msgpackmemo1arg for file 2019_07_30/get_epoch_to_domain_to_time_spent/dfeda7667999696559493b53.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_epoch_to_domain_to_time_spent/dfeda7667999696559493b53.msgpack'
performing computation 2019_07_30/get_epoch_to_domain_to_time_spent/dfeda7667999696559493b53.msgpack for arg dfeda7667999696559493b53
done with computation 2019_07_30/get_epoch_to_domain_to_time_spent/dfeda7667999696559493b53.msgpack
exception in msgpackmemo1arg for file 2019_07_30/get_epoch_to_domain_to_time_spent/e099a42d65137c47023dc4d8.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_epoch_to_domain_to_time_spent/e099a42d65137c47023dc4d8.msgpack'
performing computation 2019_07_30/get_epoch_to_domain_to_time_spent/e099a42d65137c47023dc4d8.msgpack for arg e099a42d65137c47023dc4d8
done with computation 2019_07_30/get_epoch_to_domain_to_time_spent/e099a42d65137c47023dc4d8.msgpac

In [15]:
#print(df)

In [16]:
%Rpush df

/usr/local/lib/python3.7/dist-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning:

from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.



In [17]:
%%R
library(lme4)
#library(sjPlot)
library(lmerTest)
#library(stargazer)


/usr/local/lib/python3.7/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:

载入需要的程辑包：Matrix


/usr/local/lib/python3.7/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang


/usr/local/lib/python3.7/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:


载入程辑包：‘lmerTest’



/usr/local/lib/python3.7/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:

The following object is masked from ‘package:lme4’:

    lmer



/usr/local/lib/python3.7/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:

The following object is masked from ‘package:stats’:

    step





In [18]:
# %%R

# df$user <- as.factor(df$user)
# #df$domain <- as.factor(df$domain)
# df$condition <- as.factor(df$condition)
# df$condition <- factor(df$condition, levels = c("survey_nochoice_nothing", "survey_nochoice_easy", "survey_nochoice_medium", "survey_nochoice_hard"))
# df$epoch <- as.factor(df$epoch)
# df$logtime <- as.numeric(df$logtime)
# df$time <- as.numeric(df$time)
# summary(df)



In [19]:
%%R

df$user <- as.factor(df$user)
#df$domain <- as.factor(df$domain)
df$condition <- as.factor(df$condition)
df$condition <- factor(df$condition, levels = c("survey_nochoice_nothing", "survey_nochoice_easy", "survey_nochoice_medium", "survey_nochoice_hard"))
df$epoch <- as.factor(df$epoch)
df$logtime <- as.numeric(df$logtime)
df$time <- as.numeric(df$time)
summary(df)


#results <- lmer(logtime ~ condition + (1|user), data = df)
#results <- lmer(logtime ~ condition + (1|user), data = df)
#results <- lmer(logtime ~ condition + (1|user) + (1|domain), data = df)
#show(results)
#show(summary(results))
#class(results) <- "lmerMod"
#stargazer(results)

                       user          epoch           time      
 3d8f23613f3849ed266ace58: 131   1298   : 130   Min.   :    1  
 11ae57ceae0127f030b9cb45:  32   1299   : 130   1st Qu.: 1632  
 44a79b99de35752f235f8c14:  28   1302   : 126   Median : 5058  
 72c593b5e280488490ae533c:  28   1301   : 123   Mean   : 8047  
 861400df952b797d5c27f6b3:  27   1300   : 122   3rd Qu.:10752  
 a493c0e6beeeb338edca3398:  27   1295   : 118   Max.   :83596  
 (Other)                 :2388   (Other):1912                  
    logtime                         condition  
 Min.   : 0.000   survey_nochoice_nothing:363  
 1st Qu.: 7.398   survey_nochoice_easy   :583  
 Median : 8.529   survey_nochoice_medium :375  
 Mean   : 8.143   survey_nochoice_hard   :459  
 3rd Qu.: 9.283   NA's                   :881  
 Max.   :11.334                                
                                               


In [20]:
# noexport

condition_to_lengths = {}
for condition,installs in condition_to_installs.items():
  condition_to_lengths[condition] = []
  for install in installs:
    for epoch,domain_to_time_spent in get_epoch_to_domain_to_time_spent(install).items():
      total_time_spent = sum(domain_to_time_spent.values())
      condition_to_lengths[condition].append(total_time_spent)
      #if 'www.facebook.com' not in domain_to_time_spent:
      #  continue
      #time_on_facebook = domain_to_time_spent.get('www.facebook.com', 0)
      #condition_to_lengths[condition].append(time_on_facebook)

In [21]:
# noexport
# bad


condition_to_lengths = {}
for condition,installs in condition_to_installs.items():
  condition_to_lengths[condition] = []
  for install in installs:
    for domain,intervention_to_session_lengths in get_domain_to_intervention_to_session_lengths_for_install(install).items():
      if domain != 'www.youtube.com':
        continue
      for intervention,session_lengths in intervention_to_session_lengths.items():
        for session_length in session_lengths:
          condition_to_lengths[condition].append(session_length)


exception in msgpackmemo1arg for file 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/00225f3e274bb40be3e8d77d.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/00225f3e274bb40be3e8d77d.msgpack'
performing computation 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/00225f3e274bb40be3e8d77d.msgpack for arg 00225f3e274bb40be3e8d77d
done with computation 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/00225f3e274bb40be3e8d77d.msgpack
exception in msgpackmemo1arg for file 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/0c5737eaf60340530863e305.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/0c5737eaf60340530863e305.msgpack'
performing computation 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/0c5737eaf60340530863e305.msgpack for arg 0c5737eaf60340530863e305
done with computation 2019_07_

done with computation 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/4210c9ad2c8d826b4240e589.msgpack
exception in msgpackmemo1arg for file 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/44a79b99de35752f235f8c14.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/44a79b99de35752f235f8c14.msgpack'
performing computation 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/44a79b99de35752f235f8c14.msgpack for arg 44a79b99de35752f235f8c14
done with computation 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/44a79b99de35752f235f8c14.msgpack
exception in msgpackmemo1arg for file 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/4e6a6cef78938183dbeb00a9.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/4e6a6cef78938183dbeb00a9.msgpack'
performing computation 2019_07_30/get_domain_to_intervention_to_

done with computation 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/861400df952b797d5c27f6b3.msgpack
exception in msgpackmemo1arg for file 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/90ea688d7e4c5def9a698875.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/90ea688d7e4c5def9a698875.msgpack'
performing computation 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/90ea688d7e4c5def9a698875.msgpack for arg 90ea688d7e4c5def9a698875
done with computation 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/90ea688d7e4c5def9a698875.msgpack
exception in msgpackmemo1arg for file 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/9b721228ef53a6fd1418482e.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/9b721228ef53a6fd1418482e.msgpack'
performing computation 2019_07_30/get_domain_to_intervention_to_

done with computation 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/d181845acef390690d06c3f6.msgpack
exception in msgpackmemo1arg for file 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/d4627172735b3f872ea65944.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/d4627172735b3f872ea65944.msgpack'
performing computation 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/d4627172735b3f872ea65944.msgpack for arg d4627172735b3f872ea65944
done with computation 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/d4627172735b3f872ea65944.msgpack
exception in msgpackmemo1arg for file 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/d72e6dee08aa505f9dcb9768.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/d72e6dee08aa505f9dcb9768.msgpack'
performing computation 2019_07_30/get_domain_to_intervention_to_

done with computation 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/0a02fa66d5236d68f5ad4f64.msgpack
exception in msgpackmemo1arg for file 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/0d15dce0f7a76a15f5bf6776.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/0d15dce0f7a76a15f5bf6776.msgpack'
performing computation 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/0d15dce0f7a76a15f5bf6776.msgpack for arg 0d15dce0f7a76a15f5bf6776
done with computation 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/0d15dce0f7a76a15f5bf6776.msgpack
exception in msgpackmemo1arg for file 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/1189937ade3af593673a741d.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/1189937ade3af593673a741d.msgpack'
performing computation 2019_07_30/get_domain_to_intervention_to_

done with computation 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/46b5ec70fa41264eb9969714.msgpack
exception in msgpackmemo1arg for file 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/4bba82459a0d72b55d4a28a0.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/4bba82459a0d72b55d4a28a0.msgpack'
performing computation 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/4bba82459a0d72b55d4a28a0.msgpack for arg 4bba82459a0d72b55d4a28a0
done with computation 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/4bba82459a0d72b55d4a28a0.msgpack
exception in msgpackmemo1arg for file 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/5973cebf38c0e0d89fcf0dee.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/5973cebf38c0e0d89fcf0dee.msgpack'
performing computation 2019_07_30/get_domain_to_intervention_to_

done with computation 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/8d341588192a9ff295be766e.msgpack
exception in msgpackmemo1arg for file 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/8d6bb5c4bcb83f75eeef663e.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/8d6bb5c4bcb83f75eeef663e.msgpack'
performing computation 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/8d6bb5c4bcb83f75eeef663e.msgpack for arg 8d6bb5c4bcb83f75eeef663e
done with computation 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/8d6bb5c4bcb83f75eeef663e.msgpack
exception in msgpackmemo1arg for file 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/8ef2be7b9924e0a97feb7caa.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/8ef2be7b9924e0a97feb7caa.msgpack'
performing computation 2019_07_30/get_domain_to_intervention_to_

done with computation 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/b5ceb57bec22bbb90cf3acfe.msgpack
exception in msgpackmemo1arg for file 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/bf5b8ef5089977e025c5494c.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/bf5b8ef5089977e025c5494c.msgpack'
performing computation 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/bf5b8ef5089977e025c5494c.msgpack for arg bf5b8ef5089977e025c5494c
done with computation 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/bf5b8ef5089977e025c5494c.msgpack
exception in msgpackmemo1arg for file 2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/c3ef0a1ab119c3b4a9da0ec0.msgpack
[Errno 2] 没有那个文件或目录: '2019_07_30/get_domain_to_intervention_to_session_lengths_for_install/c3ef0a1ab119c3b4a9da0ec0.msgpack'
performing computation 2019_07_30/get_domain_to_intervention_to_

In [22]:
# noexport

condition_to_lengths = {}
for condition,installs in condition_to_installs.items():
  condition_to_lengths[condition] = []
  for install in installs:
    all_lengths_for_user = []
    for epoch,domain_to_time_spent in get_epoch_to_domain_to_time_spent(install).items():
      total_time_spent = sum(domain_to_time_spent.values())
      all_lengths_for_user.append(total_time_spent)
    if len(all_lengths_for_user) == 0:
      continue
    condition_to_lengths[condition].append(np.median(all_lengths_for_user))
      #if 'www.facebook.com' not in domain_to_time_spent:
      #  continue
      #time_on_facebook = domain_to_time_spent.get('www.facebook.com', 0)
      #condition_to_lengths[condition].append(time_on_facebook)

In [23]:
# noexport

condition_to_lengths = {}
for condition,installs in condition_to_installs.items():
  condition_to_lengths[condition] = []
  for install in installs:
    for epoch,domain_to_time_spent in get_epoch_to_domain_to_time_spent(install).items():
      if 'www.facebook.com' not in domain_to_time_spent:
        continue
      time_on_facebook = domain_to_time_spent.get('www.facebook.com', 0)
      condition_to_lengths[condition].append(time_on_facebook)

In [36]:
# noexport
from scipy.stats import ttest_ind

condition_to_mlengths = {}
for condition,session_lengths in condition_to_lengths.items():
  print(condition, np.mean(session_lengths))
  condition_to_mlengths[condition] = np.mean(session_lengths) / 60

#print(ttest_ind(condition_to_lengths['survey_nochoice_hard'], condition_to_lengths['survey_nochoice_nothing']))
print(ttest_ind(condition_to_lengths['survey_nochoice_hard'], condition_to_lengths['survey_nochoice_easy']))
#print(ttest_ind(condition_to_lengths['survey_nochoice_easy'], condition_to_lengths['survey_nochoice_medium']))


survey 861.6300578034682
survey_nochoice_hard 757.9375
survey_nochoice_nothing 716.4223602484471
nodefault_forcedchoice_userchoice 550.8961038961039
survey_nochoice_easy 1860.5016835016836
survey_nochoice_medium 1332.0217391304348
Ttest_indResult(statistic=-3.318992610795198, pvalue=0.0009816290463789327)


In [37]:
from plot_utils import *


In [38]:
#from plot_utils import *
plot_dict_as_bar(condition_to_mlengths, title='Total minutes spent daily on sites', xlabel='Difficulty', ylabel='Minutes spent daily on sites', remap_labels={'survey_nochoice_nothing': 'Nothing', 'survey_nochoice_easy': 'Easy', 'survey_nochoice_medium': 'Medium', 'survey_nochoice_hard': 'Hard', 'survey': 'Experience Sampling', 'nodefault_forcedchoice_userchoice': 'Onboarding Choice'})

In [27]:
# for install in installs_with_experiment_vars:
#   #seconds_on_domain_per_session = get_collection_for_install(install, 'synced:seconds_on_domain_per_session')
#   #interventions_active_for_domain_and_session = get_collection_for_install(install, 'synced:interventions_active_for_domain_and_session')
#   res = get_domain_to_intervention_to_session_lengths_for_install(install)
#   #if len(res) > 0:
#   #  print(res)
#   #  break